In [1]:
from __future__ import division, print_function

import pandas as pd
import numpy as np
import os

# Read in Future data and split to train test sets

In [31]:
files = filter(lambda x: 'F_' in x, os.listdir('./tickerData/'))
res = []
for filename in files:
    res.append(pd.read_table('./tickerData/' + filename, sep=',', index_col='DATE',
                             parse_dates=True, skipinitialspace=True,
                             usecols=['CLOSE','OPEN','HIGH','LOW','DATE']))

In [44]:
df = pd.concat(res, axis=1,
               keys=map(lambda x: x[:-4], files),
               names=['TICKER', 'PRICE'])['1990-01-01':'2019-01-01']\
        .asfreq('D').asfreq('B')

In [48]:
base_train = df[:'2015-12-31']
base_test = df['2016-01-01':'2017-12-31']
stack_test = df['2018-01-01':]

In [49]:
base_train.shape

(6783, 352)

In [54]:
base_train.to_pickle('data_base_train.pkl')

In [50]:
base_test.shape

(521, 352)

In [56]:
base_test.to_pickle('data_base_test.pkl')

In [51]:
stack_test.shape

(261, 352)

In [57]:
stack_test.to_pickle('data_stack_test.pkl')

# Read in economic indicators and extrapolate and train test split

In [8]:
files = filter(lambda x: 'USA_' in x, os.listdir('./tickerData/'))
res = []
for filename in files:
    res.append(pd.read_table('./tickerData/' + filename, sep=',', index_col='DATE',
                             parse_dates=True, skipinitialspace=True,
                             usecols=['CLOSE','DATE']))

In [187]:
df = pd.concat(res, axis=1)['1990-01-01':'2019-01-01']
df.columns = map(lambda x: x[:-4], files)
df.head()

,USA_ADP,USA_BC,USA_BI,USA_BOT,USA_CCPI,USA_CCR,USA_CF,USA_CFNAI,USA_CHJC,USA_CINF,...,USA_PHS,USA_PP,USA_PPIC,USA_RFMI,USA_RSEA,USA_RSM,USA_RSY,USA_TVS,USA_UNR,USA_WINV
DATE,,,,,,,,,,,,,,,,,,,,,
1990-01-01,NaN,46.8,NaN,-7531.0,131.1,3.7,-8115.0,0.25,NaN,4.4,...,NaN,NaN,4.64,NaN,NaN,NaN,NaN,NaN,5.4,NaN
1990-01-31,NaN,47.4,NaN,-7631.0,131.6,4.8,15613.0,-0.03,NaN,4.4,...,NaN,NaN,4.91,NaN,NaN,NaN,NaN,NaN,5.4,NaN
1990-02-28,NaN,47.2,NaN,-4571.0,132.1,4.3,6310.0,-0.38,NaN,4.4,...,NaN,NaN,5.85,NaN,NaN,NaN,NaN,NaN,5.4,NaN
1990-04-02,NaN,49.1,NaN,-7454.0,132.7,3.1,-2468.0,0.83,NaN,4.6,...,NaN,NaN,5.10,NaN,NaN,NaN,NaN,NaN,5.3,NaN
1990-04-30,NaN,49.9,NaN,-5141.0,133.5,1.1,6192.0,0.21,NaN,4.9,...,NaN,NaN,4.55,NaN,NaN,NaN,NaN,NaN,5.2,NaN


In [44]:
names = pd.read_csv('./EconomicIndicators.csv')

In [159]:
# drop indicators that only have data from 1994 or later
cols_to_drop = df.loc[:, df.isna().sum() > 48].columns.tolist()
names[names['Code'].isin(cols_to_drop)] # just nice 10 columns

,Country,Category,Code
0,United States,ADP Employment Change,USA_ADP
1,United States,Average Hourly Earnings,USA_EARN
2,United States,Average Weekly Hours,USA_HRS
8,United States,Challenger Job Cuts,USA_CHJC
15,United States,Dallas Fed Manufacturing Index,USA_DFMI
31,United States,Job Offers,USA_JBO
40,United States,Non Manufacturing PMI,USA_NMPMI
42,United States,NY Empire State Manufacturing Index,USA_EMPST
43,United States,Pending Home Sales,USA_PHS
45,United States,Producer Prices,USA_PP


In [182]:
# fill NAs by linearly interpolating between known values
# 1 month rolling window to overcome NAs between new months
# use rolling mean of interpolated values for simplicity
# rounding to prevent overfitting of models to interpolation
indicators = df.drop(columns=cols_to_drop)\
                .asfreq('D').asfreq('B')\
                .interpolate()\
                .rolling(20, min_periods=2)\
                .mean()\
                .round(2)

In [184]:
base_train = df.drop(columns=cols_to_drop)\
                .asfreq('D').asfreq('B')\
                .loc[:'2015-12-31',:]\
                .interpolate()\
                .rolling(20, min_periods=2)\
                .mean()\
                .round(2)
base_train.to_pickle('data_indicators_base_train.pkl')

In [185]:
base_test = df.drop(columns=cols_to_drop)\
                .asfreq('D').asfreq('B')\
                .loc['2016-01-01':'2017-12-31',:]\
                .interpolate()\
                .rolling(20, min_periods=2)\
                .mean()\
                .round(2)
base_test.to_pickle('data_indicators_base_test.pkl')

In [186]:
stack_test = df.drop(columns=cols_to_drop)\
                .asfreq('D').asfreq('B')\
                .loc['2018-01-01':,:]\
                .interpolate()\
                .rolling(20, min_periods=2)\
                .mean()\
                .round(2)
stack_test.to_pickle('data_indicators_stack_test.pkl')